In [134]:
import os
import re
import pandas as pd
from IPython.display import display, HTML, Code
from IPython.display import clear_output

pd.set_option('display.max_colwidth', -1)

C:\Users\N\AppData\Local\Temp\ipykernel_7980\1432709121.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [167]:
import Tweet_Geo_constants as constants
import helper
# import LLM_Geo_kernel.Solution as Solution

from Tweet_Geo_kernel import Localization

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load tweet user infomration

In [116]:
user_CSV = r'.\test\2023-06-02T16_19_47.000Z_2023-06-03T00_57_38.000Z_499_includes_users.csv'
raw_user_df = pd.read_csv(user_CSV)

raw_user_df = raw_user_df.fillna("")
raw_user_df['description'] = raw_user_df['description'].apply(helper.clean_tweet)
raw_user_df['location'] = raw_user_df['location'].apply(helper.clean_tweet)


raw_user_df[['description', 'location']]

,description,location
0,Retired York Regional police officer living with a physical disability. I do not believe in fake science or government mandated drugs.,"Elliott Lake, Ontario, Canada"
1,Telling the other side of the story. Follow and,Canada
2,Speaker. Author My Tar Sands Tipping Point w CBC. Communications Mngr-,
3,I Can Fart On Command Community gardening is a tactic.,Desert Earth
4,Print and online journal dedicated to thoughtful journalism about environmental and other intersectional issues. Send pitches to submissions.org.,"ÜT: 37.7889,-122.401312"
...,...,...
536,,"Southern Corsica, Corsica"
537,Dartmouthian. Mother of Vincent. Multimedia journalist and producer . Personal Tweeter. Goose memorializer. Saver of bees & chairs. Totus Tuus.,"Ponamogoatitjg/ Dartmouth, NS"
538,"PhD, US history, JD. Book:","Oklahoma City, OK"
539,"Welcome to our support channel. Burning questions? Head to the Neighbourhood - Pour un soutien en français, visitez",Canada


In [140]:
user_df

,created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified
486,2015-05-05T10:04:20.000Z,,,3235466169,,Grahame Shaw,,https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png,False,"{'followers_count': 125, 'following_count': 135, 'tweet_count': 33338, 'listed_count': 2}",,ShawgrahameShaw,False
60,2021-03-24T23:15:52.000Z,Orgullosamente Veracruzana ❤️🇲🇽 De regreso amigos 💃 I am backkkkkkkkk. (BARRENECHEA),,1374862493462396932,México 🇲🇽,Pattygb 🇲🇽,,https://pbs.twimg.com/profile_images/1374868776139714563/6eDVqlju_normal.jpg,False,"{'followers_count': 1354, 'following_count': 2112, 'tweet_count': 99613, 'listed_count': 2}",,Pattygb5,False
525,2023-03-31T18:55:00.000Z,,,1641876586579255322,,tassan dominique,,https://pbs.twimg.com/profile_images/1642123819073654786/ktNi3TNd_normal.jpg,False,"{'followers_count': 12, 'following_count': 67, 'tweet_count': 132, 'listed_count': 0}",,TassanDominique,False


In [169]:
user_df = raw_user_df.fillna("").sample(3)


print(tweet_loc.prompt_for_user_localization)
# for idx, row in user_df.iloc[:4].iterrows():
model=r"gpt-3.5-turbo-0613"
# model=r"gpt-3.5-turbo"

# model=r"gpt-4"

tweet_loc = Localization(user_df=user_df,
                        model=model)


response_for_user_loc = tweet_loc.get_LLM_user_loc_response() 
response_for_user_loc

Your role: A professional Geo-information scientist and developer good at Python. 
Your task: Extract Twitter user’s home location based on their profile description and self-reported home location.  
Requirement:
1. The input is a list of dictionary elements, e.g.: [{"description": "Heart of the big apple.", "location:": "New York City"}]
2. The input is a list of dictionary elements, e.g.: [{"location": "New York City"}]
3. Make sure the output element count is the same as the input. If you cannot find or determine a location, return {"location": ""} for that tweet.
4. If the description contradict to the self-reported home location, return the self-reported home location.
5. If the location is longitude and latitude, return them as "longitude, latitude".
6. Keep the original language of extracted location name.
7. If the extracted location is a nick, return the standard name for accessbility, such as return "New York City" for "Big apple". 
List of user discription and self-report h

[<OpenAIObject chat.completion.chunk id=chatcmpl-7SGYdzgo7AME9av1HWSOOu4JOLkse at 0x1ba7d2e8810> JSON: {
   "choices": [
     {
       "delta": {
         "content": "",
         "role": "assistant"
       },
       "finish_reason": null,
       "index": 0
     }
   ],
   "created": 1686971187,
   "id": "chatcmpl-7SGYdzgo7AME9av1HWSOOu4JOLkse",
   "model": "gpt-3.5-turbo-0613",
   "object": "chat.completion.chunk"
 },
 <OpenAIObject chat.completion.chunk id=chatcmpl-7SGYdzgo7AME9av1HWSOOu4JOLkse at 0x1ba0049b560> JSON: {
   "choices": [
     {
       "delta": {
         "content": "import"
       },
       "finish_reason": null,
       "index": 0
     }
   ],
   "created": 1686971187,
   "id": "chatcmpl-7SGYdzgo7AME9av1HWSOOu4JOLkse",
   "model": "gpt-3.5-turbo-0613",
   "object": "chat.completion.chunk"
 },
 <OpenAIObject chat.completion.chunk id=chatcmpl-7SGYdzgo7AME9av1HWSOOu4JOLkse at 0x1ba0049b7e0> JSON: {
   "choices": [
     {
       "delta": {
         "content": " ge"
       }

In [159]:

for idx, row in user_df.iloc[:4].iterrows():
    # print(idx, row)
    userID = row['id']
    raw_description = row['description']
    raw_location = row['location']
    
    print(userID, raw_description, raw_location)
    description = helper.clean_tweet(raw_description)
    location = helper.clean_tweet(raw_location)
    print(userID, description, location)
    print()
    
    list_of_dict = user_df[['description', 'location']].to_dict('records')
    
list_of_dict

1223950760 Visit  & sign up to become a #LPC supporter, to donate or to become a volunteer. Kildonan-St. Paul Winnipeg MB
1223950760 Visit  & sign up to become a #LPC supporter, to donate or to become a volunteer. Kildonan-St. Paul Winnipeg MB

535913183 Official tweets of The Day Trading Academy.  Where traders find success, wealth, or freedom through #investing and #trading.  Let's Create Your Lifestyle. Worldwide
535913183 Official tweets of The Day Trading Academy.  Where traders find success, wealth, or freedom through #investing and #trading.  Let's Create Your Lifestyle. Worldwide

1569148495218593792 🇨🇦gal🏋🏼‍♀️🚴‍♀️😷 Vaccines=Science=Life Proud book🪱 #Dendrophile  #SpaceJunkie  #SlavaUkraïni🇺🇦        Don't argue w/ morons. Anishinabe Algonquin territory
1569148495218593792 🇨🇦gal🏋🏼‍♀️🚴‍♀️😷 Vaccines=Science=Life Proud book🪱 #Dendrophile  #SpaceJunkie  #SlavaUkraïni🇺🇦        Don't argue w/ morons. Anishinabe Algonquin territory



[{'description': 'Visit  & sign up to become a #LPC supporter, to donate or to become a volunteer.',
  'location': 'Kildonan-St. Paul Winnipeg MB'},
 {'description': "Official tweets of The Day Trading Academy.  Where traders find success, wealth, or freedom through #investing and #trading.  Let's Create Your Lifestyle.",
  'location': 'Worldwide'},
 {'description': "🇨🇦gal🏋🏼\u200d♀️🚴\u200d♀️😷 Vaccines=Science=Life Proud book🪱 #Dendrophile  #SpaceJunkie  #SlavaUkraïni🇺🇦        Don't argue w/ morons.",
  'location': 'Anishinabe Algonquin territory'}]

In [75]:
def clean_tweet(tweet_text):
    # Remove URLs
    tweet_text = re.sub(r'http\S+|www.\S+', '', tweet_text, flags=re.MULTILINE)
    # Remove usernames/mentions
    tweet_text = re.sub(r'@\w+[ ,]*', '', tweet_text)
    return tweet_text

    # Example usage:
tweet = "@user, @user2, @user3 This, is a tweet with a URL: http://example.com, https://example.com"
print(clean_tweet(tweet))

This, is a tweet with a URL:  
